In [1]:
# Standard imports
import os
import torch
from tqdm import tqdm
import plotly.express as px
import pandas as pd
# Imports for displaying vis in Colab / notebook

torch.set_grad_enabled(False)

# For the most part I'll try to import functions and classes near where they are used
# to make it clear where they come from.

if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")

Device: cuda


In [4]:
from IPython.display import IFrame
html_template = "https://neuronpedia.org/{}/{}/{}?embed=true&embedexplanation=true&embedplots=true&embedtest=true&height=300"

def get_dashboard_html(sae_release = "gemma-2-2b", sae_id="8-gemmascope-res-16k", feature_idx=0):
    return html_template.format(sae_release, sae_id, feature_idx)

In [2]:
# from transformer_lens import HookedTransformer
from sae_lens import SAE, HookedSAETransformer
os.environ["HF_TOKEN"] = "<hf token here>"
model = HookedSAETransformer.from_pretrained("google/gemma-2-2b", device = device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2-2b into HookedTransformer


In [56]:
from transformer_lens.utils import test_prompt

prompt = "What is the output of 53 plus 32 ? A: "
answer = '8'
# Show that the model can confidently predict the next token.
test_prompt(prompt, answer, model)

Tokenized prompt: ['<bos>', 'What', ' is', ' the', ' output', ' of', ' ', '5', '3', ' plus', ' ', '3', '2', ' ?', ' A', ':', ' ']
Tokenized answer: [' ', '8']


Performance on answer token:
Rank: 469      Logit: 10.61 Prob:  0.00% Token: | |

Top 0th token. Logit: 28.92 Prob: 40.73% Token: |8|
Top 1th token. Logit: 28.03 Prob: 16.72% Token: |5|
Top 2th token. Logit: 27.64 Prob: 11.27% Token: |1|
Top 3th token. Logit: 27.49 Prob:  9.76% Token: |2|
Top 4th token. Logit: 26.98 Prob:  5.83% Token: |3|
Top 5th token. Logit: 26.85 Prob:  5.16% Token: |7|
Top 6th token. Logit: 26.71 Prob:  4.45% Token: |4|
Top 7th token. Logit: 26.51 Prob:  3.67% Token: |6|
Top 8th token. Logit: 25.52 Prob:  1.36% Token: |9|
Top 9th token. Logit: 25.19 Prob:  0.98% Token: |0|


Performance on answer token:
Rank: 0        Logit: 28.33 Prob: 29.65% Token: |8|

Top 0th token. Logit: 28.33 Prob: 29.65% Token: |8|
Top 1th token. Logit: 28.08 Prob: 23.24% Token: |5|
Top 2th token. Logit: 27.49 Prob: 12.86% Token: |1|
Top 3th token. Logit: 27.20 Prob:  9.60% Token: |2|
Top 4th token. Logit: 26.87 Prob:  6.89% Token: |3|
Top 5th token. Logit: 26.57 Prob:  5.13% Token: |7|
Top 6th token. Logit: 26.47 Prob:  4.64% Token: |6|
Top 7th token. Logit: 26.46 Prob:  4.57% Token: |4|
Top 8th token. Logit: 25.64 Prob:  2.02% Token: |9|
Top 9th token. Logit: 25.09 Prob:  1.17% Token: |0|


Ranks of the answer tokens: [(' ', 469), ('8', 0)]

In [41]:
from transformer_lens.utils import test_prompt

prompt = "What is 73 plus 19 ? A: <strong>"
answer = '8'
# Show that the model can confidently predict the next token.
test_prompt(prompt, answer, model)

Tokenized prompt: ['<bos>', 'What', ' is', ' ', '7', '3', ' plus', ' ', '1', '9', ' ?', ' A', ':', ' ', '<strong>']
Tokenized answer: [' ', '8']


Performance on answer token:
Rank: 16       Logit: 22.02 Prob:  0.28% Token: | |

Top 0th token. Logit: 27.58 Prob: 73.61% Token: |9|
Top 1th token. Logit: 24.96 Prob:  5.36% Token: |8|
Top 2th token. Logit: 24.69 Prob:  4.12% Token: |1|
Top 3th token. Logit: 24.10 Prob:  2.28% Token: |7|
Top 4th token. Logit: 23.86 Prob:  1.79% Token: |Nin|
Top 5th token. Logit: 23.65 Prob:  1.44% Token: |nine|
Top 6th token. Logit: 23.58 Prob:  1.35% Token: |2|
Top 7th token. Logit: 23.15 Prob:  0.88% Token: |3|
Top 8th token. Logit: 22.86 Prob:  0.65% Token: |6|
Top 9th token. Logit: 22.70 Prob:  0.56% Token: |<em>|


Performance on answer token:
Rank: 1        Logit: 24.50 Prob: 13.95% Token: |8|

Top 0th token. Logit: 25.97 Prob: 60.47% Token: |9|
Top 1th token. Logit: 24.50 Prob: 13.95% Token: |8|
Top 2th token. Logit: 23.69 Prob:  6.18% Token: |</strong>|
Top 3th token. Logit: 23.61 Prob:  5.75% Token: |1|
Top 4th token. Logit: 23.22 Prob:  3.88% Token: |7|
Top 5th token. Logit: 22.89 Prob:  2.79% Token: |2|
Top 6th token. Logit: 22.38 Prob:  1.68% Token: |3|
Top 7th token. Logit: 22.11 Prob:  1.28% Token: |6|
Top 8th token. Logit: 21.93 Prob:  1.07% Token: |5|
Top 9th token. Logit: 21.72 Prob:  0.87% Token: |<em>|


Ranks of the answer tokens: [(' ', 16), ('8', 1)]

In [29]:
layer = 10
sae, cfg_dict, sparsity = SAE.from_pretrained(
        release="gemma-scope-2b-pt-res-canonical",  # <- Release name 
        sae_id=f"layer_{layer}/width_16k/canonical",  # <- SAE id (not always a hook point!)
        device=device
    )

In [57]:
from dataclasses import dataclass
from functools import partial
from typing import Any, Literal, NamedTuple, Callable

import torch
from sae_lens import SAE
from transformer_lens import HookedTransformer
from transformer_lens.hook_points import HookPoint
import torch.nn.functional as F

# Use the functional API with inplace=False
# feature_acts = self.hook_sae_acts_post(F.relu(hidden_pre, inplace=False))

class SaeReconstructionCache(NamedTuple):
    sae_in: torch.Tensor
    feature_acts: torch.Tensor
    sae_out: torch.Tensor
    sae_error: torch.Tensor


def track_grad(tensor: torch.Tensor) -> None:
    """wrapper around requires_grad and retain_grad"""
    tensor.requires_grad_(True)
    tensor.retain_grad()


@dataclass
class ApplySaesAndRunOutput:
    model_output: torch.Tensor
    model_activations: dict[str, torch.Tensor]
    sae_activations: dict[str, SaeReconstructionCache]

    def zero_grad(self) -> None:
        """Helper to zero grad all tensors in this object."""
        self.model_output.grad = None
        for act in self.model_activations.values():
            act.grad = None
        for cache in self.sae_activations.values():
            cache.sae_in.grad = None
            cache.feature_acts.grad = None
            cache.sae_out.grad = None
            cache.sae_error.grad = None


def apply_saes_and_run(
    model: HookedTransformer,
    saes: dict[str, SAE],
    input: Any,
    include_error_term: bool = True,
    track_model_hooks: list[str] | None = None,
    return_type: Literal["logits", "loss"] = "logits",
    track_grads: bool = False,
) -> ApplySaesAndRunOutput:
    """
    Apply the SAEs to the model at the specific hook points, and run the model.
    By default, this will include a SAE error term which guarantees that the SAE
    will not affect model output. This function is designed to work correctly with
    backprop as well, so it can be used for gradient-based feature attribution.

    Args:
        model: the model to run
        saes: the SAEs to apply
        input: the input to the model
        include_error_term: whether to include the SAE error term to ensure the SAE doesn't affect model output. Default True
        track_model_hooks: a list of hook points to record the activations and gradients. Default None
        return_type: this is passed to the model.run_with_hooks function. Default "logits"
        track_grads: whether to track gradients. Default False
    """

    fwd_hooks = []
    bwd_hooks = []

    sae_activations: dict[str, SaeReconstructionCache] = {}
    model_activations: dict[str, torch.Tensor] = {}

    # this hook just track the SAE input, output, features, and error. If `track_grads=True`, it also ensures
    # that requires_grad is set to True and retain_grad is called for intermediate values.
    def reconstruction_hook(sae_in: torch.Tensor, hook: HookPoint, hook_point: str):  # noqa: ARG001
        sae = saes[hook_point]
#         x = sae_in.to(sae.dtype)
#         x = sae.reshape_fn_in(x)
#         x = sae.hook_sae_input(x)
#         x = sae.run_time_activation_norm_fn_in(x)

#         sae_in = x - (sae.b_dec * sae.cfg.apply_b_dec_to_input)

#         # # "... d_in, d_in d_sae -> ... d_sae",
#         hidden_pre = sae.hook_sae_acts_pre(sae_in @ sae.W_enc + sae.b_enc)
#         feature_acts = sae.hook_sae_acts_post(F.relu(hidden_pre, inplace=False))
        feature_acts = sae.encode(sae_in)
        
        
        sae_out = sae.decode(feature_acts)
        sae_error = (sae_in - sae_out).detach().clone()
        if track_grads:
            track_grad(sae_error)
            track_grad(sae_out)
            track_grad(feature_acts)
            track_grad(sae_in)
        sae_activations[hook_point] = SaeReconstructionCache(
            sae_in=sae_in,
            feature_acts=feature_acts,
            sae_out=sae_out,
            sae_error=sae_error,
        )

        if include_error_term:
            return sae_out + sae_error
        return sae_out

    def sae_bwd_hook(output_grads: torch.Tensor, hook: HookPoint):  # noqa: ARG001
        # this just passes the output grads to the input, so the SAE gets the same grads despite the error term hackery
        return (output_grads,)

    # this hook just records model activations, and ensures that intermediate activations have gradient tracking turned on if needed
    def tracking_hook(hook_input: torch.Tensor, hook: HookPoint, hook_point: str):  # noqa: ARG001
        model_activations[hook_point] = hook_input
        if track_grads:
            track_grad(hook_input)
        return hook_input

    for hook_point in saes.keys():
        fwd_hooks.append(
            (hook_point, partial(reconstruction_hook, hook_point=hook_point))
        )
        bwd_hooks.append((hook_point, sae_bwd_hook))
    for hook_point in track_model_hooks or []:
        fwd_hooks.append((hook_point, partial(tracking_hook, hook_point=hook_point)))

    # now, just run the model while applying the hooks
    with model.hooks(fwd_hooks=fwd_hooks, bwd_hooks=bwd_hooks):
        model_output = model(input, return_type=return_type)

    return ApplySaesAndRunOutput(
        model_output=model_output,
        model_activations=model_activations,
        sae_activations=sae_activations,
    )


from dataclasses import dataclass
from transformer_lens.hook_points import HookPoint
from dataclasses import dataclass
from functools import partial
from typing import Any, Literal, NamedTuple

import torch
from sae_lens import SAE
from transformer_lens import HookedTransformer
from transformer_lens.hook_points import HookPoint
torch.autograd.set_detect_anomaly(True)
EPS = 1e-8

torch.set_grad_enabled(True)
@dataclass
class AttributionGrads:
    metric: torch.Tensor
    model_output: torch.Tensor
    model_activations: dict[str, torch.Tensor]
    sae_activations: dict[str, SaeReconstructionCache]


@dataclass
class Attribution:
    model_attributions: dict[str, torch.Tensor]
    model_activations: dict[str, torch.Tensor]
    model_grads: dict[str, torch.Tensor]
    sae_feature_attributions: dict[str, torch.Tensor]
    sae_feature_activations: dict[str, torch.Tensor]
    sae_feature_grads: dict[str, torch.Tensor]
    sae_errors_attribution_proportion: dict[str, float]


def calculate_attribution_grads(
    model: HookedSAETransformer,
    prompt: str,
    metric_fn: Callable[[torch.Tensor], torch.Tensor],
    track_hook_points: list[str] | None = None,
    include_saes: dict[str, SAE] | None = None,
    return_logits: bool = True,
    include_error_term: bool = True,
) -> AttributionGrads:
    """
    Wrapper around apply_saes_and_run that calculates gradients wrt to the metric_fn.
    Tracks grads for both SAE feature and model neurons, and returns them in a structured format.
    """
    output = apply_saes_and_run(
        model,
        saes=include_saes or {},
        input=prompt,
        return_type="logits" if return_logits else "loss",
        track_model_hooks=track_hook_points,
        include_error_term=include_error_term,
        track_grads=True,
    )
    metric = metric_fn(output.model_output)
    output.zero_grad()
    metric.backward()
    return AttributionGrads(
        metric=metric,
        model_output=output.model_output,
        model_activations=output.model_activations,
        sae_activations=output.sae_activations,
    )


def calculate_feature_attribution(
    model: HookedSAETransformer,
    input: Any,
    metric_fn: Callable[[torch.Tensor], torch.Tensor],
    track_hook_points: list[str] | None = None,
    include_saes: dict[str, SAE] | None = None,
    return_logits: bool = True,
    include_error_term: bool = True,
) -> Attribution:
    """
    Calculate feature attribution for SAE features and model neurons following
    the procedure in https://transformer-circuits.pub/2024/march-update/index.html#feature-heads.
    This include the SAE error term by default, so inserting the SAE into the calculation is
    guaranteed to not affect the model output. This can be disabled by setting `include_error_term=False`.

    Args:
        model: The model to calculate feature attribution for.
        input: The input to the model.
        metric_fn: A function that takes the model output and returns a scalar metric.
        track_hook_points: A list of model hook points to track activations for, if desired
        include_saes: A dictionary of SAEs to include in the calculation. The key is the hook point to apply the SAE to.
        return_logits: Whether to return the model logits or loss. This is passed to TLens, so should match whatever the metric_fn expects (probably logits)
        include_error_term: Whether to include the SAE error term in the calculation. This is recommended, as it ensures that the SAE will not affecting the model output.
    """
    # first, calculate gradients wrt to the metric_fn.
    # these will be multiplied with the activation values to get the attributions
    outputs_with_grads = calculate_attribution_grads(
        model,
        input,
        metric_fn,
        track_hook_points,
        include_saes=include_saes,
        return_logits=return_logits,
        include_error_term=include_error_term,
    )
    model_attributions = {}
    model_activations = {}
    model_grads = {}
    sae_feature_attributions = {}
    sae_feature_activations = {}
    sae_feature_grads = {}
    sae_error_proportions = {}
    # this code is long, but all it's doing is multiplying the grads by the activations
    # and recording grads, acts, and attributions in dictionaries to return to the user
    with torch.no_grad():
        for name, act in outputs_with_grads.model_activations.items():
            assert act.grad is not None
            raw_activation = act.detach().clone()
            model_attributions[name] = (act.grad * raw_activation).detach().clone()
            model_activations[name] = raw_activation
            model_grads[name] = act.grad.detach().clone()
        for name, act in outputs_with_grads.sae_activations.items():
            assert act.feature_acts.grad is not None
            assert act.sae_out.grad is not None
            raw_activation = act.feature_acts.detach().clone()
            sae_feature_attributions[name] = (
                (act.feature_acts.grad * raw_activation).detach().clone()
            )
            sae_feature_activations[name] = raw_activation
            sae_feature_grads[name] = act.feature_acts.grad.detach().clone()
            if include_error_term:
                assert act.sae_error.grad is not None
                error_grad_norm = act.sae_error.grad.norm().item()
            else:
                error_grad_norm = 0
            sae_out_norm = act.sae_out.grad.norm().item()
            sae_error_proportions[name] = error_grad_norm / (
                sae_out_norm + error_grad_norm + EPS
            )
        return Attribution(
            model_attributions=model_attributions,
            model_activations=model_activations,
            model_grads=model_grads,
            sae_feature_attributions=sae_feature_attributions,
            sae_feature_activations=sae_feature_activations,
            sae_feature_grads=sae_feature_grads,
            sae_errors_attribution_proportion=sae_error_proportions,
        )
        
        
# prompt = " Tiger Woods plays the sport of"
# pos_token = model.tokenizer.encode(" golf")[0]
prompt = "What is the output of 53 plus 32 ? A: "
pos_token = [model.tokenizer.encode("8")[1]]
neg_token = [model.tokenizer.encode("2")[1]]
# def metric_fn(logits: torch.tensor, pos_token: torch.tensor =pos_token, neg_token: torch.Tensor=neg_token) -> torch.Tensor:
#     return logits[0,-1,pos_token] - logits[0,-1,neg_token]

def metric_fn(logits: torch.Tensor, pos_token: torch.Tensor = pos_token, neg_token: torch.Tensor = neg_token) -> torch.Tensor:
    return (logits[0, -1, pos_token] - logits[0, -1, neg_token]).sum()


feature_attribution_df = calculate_feature_attribution(
    input = prompt,
    model = model,
    metric_fn = metric_fn,
    include_saes={sae.cfg.hook_name: sae},
    include_error_term=True,
    return_logits=True,
)


In [58]:
[model.tokenizer.encode("is")[1]]

[502]

In [49]:
# Generate the plot
tokens = model.to_str_tokens(prompt)
unique_tokens = [f"{i}/{t}" for i, t in enumerate(tokens)]

fig = px.bar(x=unique_tokens,
             y=feature_attribution_df.sae_feature_attributions[sae.cfg.hook_name][0].sum(-1).detach().cpu().numpy())

# Show the plot
fig.show()

# Save the plot as a PNG file
fig.write_image("feature_attribution_plot.png")

In [51]:
def convert_sparse_feature_to_long_df(sparse_tensor: torch.Tensor) -> pd.DataFrame:
    """
    Convert a sparse tensor to a long format pandas DataFrame.
    """
    df = pd.DataFrame(sparse_tensor.detach().cpu().numpy())
    df_long = df.melt(ignore_index=False, var_name='column', value_name='value')
    df_long.columns = ["feature", "attribution"]
    df_long_nonzero = df_long[df_long['attribution'] != 0]
    df_long_nonzero = df_long_nonzero.reset_index().rename(columns={'index': 'position'})
    return df_long_nonzero

df_long_nonzero = convert_sparse_feature_to_long_df(feature_attribution_df.sae_feature_attributions[sae.cfg.hook_name][0])
df_long_nonzero.sort_values("attribution", ascending=False).head(30)

,position,feature,attribution
3976,16,9822,0.667203
5299,0,13146,0.341047
1249,16,3078,0.273905
2881,0,7019,0.256868
4740,13,11772,0.221892
2182,11,5256,0.161158
4743,16,11772,0.157880
835,8,2078,0.157479
1639,13,3986,0.123287
5057,0,12541,0.093872


In [52]:
for pos in [7, 11]: #range(6, 17):
    for i, v in df_long_nonzero.query(f"position=={pos}").groupby("feature").attribution.sum().sort_values(ascending=False).head(5).items():
        print(f"Pos {pos}, Feature {i} had a total attribution of {v:.2f}")
        html = get_dashboard_html(sae_release = "gemma-2-2b", sae_id="10-gemmascope-res-16k", feature_idx=int(i))
        display(IFrame(html, width=1200, height=300))

Pos 7, Feature 3986 had a total attribution of 0.03


Pos 7, Feature 10583 had a total attribution of 0.03


Pos 7, Feature 3406 had a total attribution of 0.02


Pos 7, Feature 1833 had a total attribution of 0.02


Pos 7, Feature 389 had a total attribution of 0.01


Pos 11, Feature 5256 had a total attribution of 0.16


Pos 11, Feature 11335 had a total attribution of 0.07


Pos 11, Feature 12035 had a total attribution of 0.05


Pos 11, Feature 15537 had a total attribution of 0.04


Pos 11, Feature 1740 had a total attribution of 0.04


In [55]:
for i, v in df_long_nonzero.groupby("feature").attribution.sum().sort_values(ascending=False).head(5).items():
    print(f"Feature {i} had a total attribution of {v:.2f}")
    html = get_dashboard_html(sae_release = "gemma-2-2b", sae_id="10-gemmascope-res-16k", feature_idx=int(i))
    display(IFrame(html, width=1200, height=300))

Feature 9822 had a total attribution of 0.65


Feature 11772 had a total attribution of 0.45


Feature 13146 had a total attribution of 0.40


Feature 3078 had a total attribution of 0.27


Feature 7019 had a total attribution of 0.26
